In [1]:
import numpy as np
from glob import glob
import tensorflow as tf
import keras
from keras.models import Model, Sequential
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Input, Lambda, Dense, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img

In [8]:
print("Tensorflow version: ", tf.__version__)
print("Keras version: ", keras.__version__)

Tensorflow version:  2.14.0
Keras version:  2.14.0


In [9]:
def get_class_name(folder_names) -> list :
  total_class = []
  for i in folder_names:
    b = i.split('_')
    total_class.append(' '.join(b[4:]))
  return total_class

train_dir = '/content/drive/MyDrive/Project_Sem5/Dataset/Potato/train'
test_dir = '/content/drive/MyDrive/Project_Sem5/Dataset/Potato/test'
valid_dir = '/content/drive/MyDrive/Project_Sem5/Dataset/Potato/valid'

folder_names = glob(train_dir+'/*')
total_class = get_class_name(folder_names)

In [10]:
total_class

['healthy', 'Early blight', 'Late blight']

In [11]:
# Image size
IMAGE_SIZE = [256, 256]

# downloading the model
base_model = InceptionV3(input_shape= IMAGE_SIZE + [3], weights='imagenet', include_top=False)
for layer in base_model.layers:
  layer.trainable=False

# Flatting the output of the last layer
x = Flatten()(base_model.output)

pred = Dense(len(total_class), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=pred)

model.summary()

87910968/87910968 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 127, 127, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 127, 127, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     

In [13]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256,256),
    batch_size=16,
    class_mode='categorical'
  )

Found 1720 images belonging to 3 classes.


In [15]:
valid_set = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(256,256),
    batch_size=16,
    class_mode='categorical'
  )

Found 215 images belonging to 3 classes.


In [16]:
test_set = test_datagen.flow_from_directory(
    valid_dir,
    target_size=(256,256),
    batch_size=16,
    class_mode='categorical'
  )

Found 215 images belonging to 3 classes.


In [2]:
%cd /content/drive/MyDrive/Project_Sem5/Dataset/Potato
%pwd

/content/drive/MyDrive/Project_Sem5/Dataset/Potato


'/content/drive/MyDrive/Project_Sem5/Dataset/Potato'

In [ ]:
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="training/model",
    save_weights_only=False,
    monitor="val_loss",
    mode="min",
    verbose=1,
    save_best_only=True
)

In [ ]:
model.fit(
    train_set,
    validation_data=valid_set,
    epochs=30,
    steps_per_epoch=len(train_set),
    validation_steps=len(valid_set),
    callbacks=[checkpoint_callback]
)

Epoch 1/30
108/108 [==============================] - ETA: 0s - loss: 2.0097 - accuracy: 0.8285
Epoch 1: val_loss improved from inf to 1.65817, saving model to training/model
108/108 [==============================] - 451s 4s/step - loss: 2.0097 - accuracy: 0.8285 - val_loss: 1.6582 - val_accuracy: 0.8605
Epoch 2/30
108/108 [==============================] - ETA: 0s - loss: 0.9053 - accuracy: 0.9122
Epoch 2: val_loss did not improve from 1.65817
108/108 [==============================] - 33s 304ms/step - loss: 0.9053 - accuracy: 0.9122 - val_loss: 1.9142 - val_accuracy: 0.8372
Epoch 3/30
108/108 [==============================] - ETA: 0s - loss: 0.8204 - accuracy: 0.9302
Epoch 3: val_loss improved from 1.65817 to 0.88585, saving model to training/model
108/108 [==============================] - 58s 541ms/step - loss: 0.8204 - accuracy: 0.9302 - val_loss: 0.8859 - val_accuracy: 0.9209
Epoch 4/30
108/108 [==============================] - ETA: 0s - loss: 0.4176 - accuracy: 0.9535
Epoch 4

In [17]:
model = keras.models.load_model("./training/model")

In [ ]:
model_save_path = '/content/drive/MyDrive/Project_Sem5/Dataset/Potato'
model.save(model_save_path+'/potato.h5')
model.save(model_save_path+'/potato2.keras')

In [18]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256,256),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

pred = model.predict(test_set)

evaluation = model.evaluate(test_set)

# Print the evaluation metrics
print("Test loss:", evaluation[0])
print("Test accuracy:", evaluation[1])

Found 214 images belonging to 3 classes.
14/14 [==============================] - 2s 58ms/step - loss: 0.5530 - accuracy: 0.9533
Test loss: 0.5530233979225159
Test accuracy: 0.9532710313796997
